We are interested in performing the following inference:

$P(\theta \; | y_{obs}) \propto \pi(\theta)P(y_{obs}\; | \theta ) $

However, calculating the liklihood is too expensive for our use case. Instead, we'd like to use and emultor to do:

$P(\theta \; | y_{obs}) \propto \pi(\theta) \int{P(y_{obs} \; | \hat{y} ) \; P(\hat{y} \;| \theta)\; d\hat{y} }$

Where $P(\hat{y} \; | \theta)$ is the emulator. We have chosen a Gaussian process emulator, which has a few additional benefits. For one, it produces an estimator $\hat{y}$ with a well-defined Gaussian distribution given its training. If our measurment errors are also Gaussian, then the above integral is a product of Gaussians which is itself a Gaussian with the below properties.

$\mathcal{N}(a,A) \cdot \mathcal{N}(b,B) = z\; \mathcal{N}(c, C)$

$C \equiv (A^{-1} + B^{-1})^{-1}$

$c \equiv C(A^{-1}a + B^{-1}b)$

And $z$ is a complex function I will define below, as it requires more explanation. 

The above relation is defined if both distributions have the same random variable. We have to perform a change of variables, then. We note that:

$P(y_{obs}\; | \hat{y}) = \frac{exp[(y_{obs}-\hat{y})^T\Sigma^{-1}(y_{obs}-\hat{y})]}{Z} = P(\hat{y}\; | y_{obs})$

Just like that, we've got:

$P(\theta \; | y_{obs}) \propto \pi(\theta) \int{P(\hat{y} \; | y_{obs} ) \; P(\hat{y} \;| \theta)\; d\hat{y} }\\ \\
P(\theta \; | y_{obs}) \propto \pi(\theta)\; z\; \int{P(\hat{y} \; | y_{obs}, \theta ) d\hat{y} } \\ \\
P(\theta \; | y_{obs}) \propto \pi(\theta)\; z\;(1)$



Now, lets revisit $z$. $z$ is effectively $P(y_{obs} | \theta)$. This is convenient, because $z$ is a random variable that is Gaussian in both $y_{obs}$ and $\bar{y}$, the mean of the distribution from the emulator, a function of theta. So,

$P(\theta \; | y_{obs}) \propto \pi(\theta)P(y_{obs}\; | \theta )\\ $

Where 

$y_{obs} | \theta \sim \mathcal{N}(D\;(T^{-1}\; C\; G^{-1} ) \; \bar{y}, D)$

And the matrices are defined as follows:

$ \\ \\T$ is the measurement eror covariance matrix. 

$ \\ \\G(\theta)$ is the covariance of $\hat{y}$ provided by the emulator. 

$\\ \\C(\theta) = (T^{-1} + G(\theta)^{-1})^{-1}$, which is the covariance of the product distribution. 
And finally, 

$\\ \\D(\theta) = (T^{-1} \; C(\theta) \; T^{-1})^{-1} = T\;(T^{-1} + G(\theta)^{-1}) \; T = T+ T\; G(\theta)^{-1}\;T$

Where all functions of $\theta$ have been explicitly shown for clarity. Below, I will attempt to implement this using my emulation software, `pearce`. 

In [52]:
from pearce.emulator.emu import OriginalRecipe, ExtraCrispy
from pearce.emulator.trainingData import parameter, PARAMS

In [53]:
import numpy as np
from os import path

In [54]:
training_dir = '/u/ki/swmclau2/des/PearceLHC/'
or_params = PARAMS[:]
#or_params.append(parameter('r', 0, 1)) #95% sure bounds aren't used for r
#emu = OriginalRecipe(training_dir,or_params)
emu = ExtraCrispy(training_dir, or_params)

In [55]:
emulation_point = [('f_c', 0.233), ('logM0', 12.0), ('sigma_logM', 0.533), 
                   ('alpha', 1.083),('logM1', 13.5), ('logMmin', 12.233)]
em_params = dict(emulation_point)

In [56]:
save_dir = '/u/ki/swmclau2/des/EmulatorMCMC/'
rpoints = np.loadtxt(path.join(save_dir, 'rpoints.npy'))
y = np.loadtxt(path.join(save_dir, 'xi.npy'))
cov = np.loadtxt(path.join(save_dir, 'cov.npy'))
T = cov #np.diag(np.diag(cov))
T = T/(np.outer(y,y)*np.log(10)**2)
#T = np.diag(np.diag(T))
y = np.log10(y)

In [57]:
from scipy.linalg import inv

In [58]:
#em_params = {p.name: theta for p in self.ordered_params}

#using my own notation
y_bar, G = emu.emulate_wrt_r(em_params, rpoints)
#y_bar, G = emu.emulate(em_params)
#y_bar = 10**y_bar
#G = G*np.outer(y_bar, y_bar)*np.log(10)**2
#G = np.diag(np.diag(G))

(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)
(1,) (1, 1) (1, 6) (496,)


In [59]:
print y_bar

[[ 3.33857219  2.93358166  2.48636433  2.21810416  1.86595312  1.48862137
   1.09563825  0.75276058  0.48297387  0.25980587  0.02334519 -0.20475356
  -0.47153732 -0.73603583 -1.04181694 -1.39814147 -1.95044229]]


In [60]:
print y

[ 3.32086144  2.58650472  2.20321426  1.89321609  1.54738839  1.20503721
  0.90378299  0.64283911  0.38615689  0.15954921 -0.06179825 -0.29184025
 -0.55316333 -0.8260194  -1.12644251 -1.4881848  -1.99929647]


In [61]:
print G.shape

(1, 1, 17)


In [62]:
G = np.diag(G[0,0,:])

In [63]:
print T

[[  1.98492892e-04  -1.94664204e-07  -3.75336378e-06   2.00431288e-06
    2.54273749e-06   1.76112827e-06   8.36316334e-07   4.48614558e-07
    4.44565515e-07   1.73739716e-06   2.90347113e-07   5.31210312e-07
   -8.97340750e-07  -7.78689621e-07   1.18011343e-06   9.96475534e-07
    1.51411906e-05]
 [ -1.94664204e-07   1.28876153e-05  -1.83466135e-06   2.98981615e-07
   -4.89438357e-09   3.43916484e-07   2.99135713e-07   2.52340019e-07
    3.43144406e-07   5.37760868e-07   5.34333892e-07   7.50863848e-07
    6.84598938e-07   1.13466742e-06   2.27919117e-06   2.27056636e-06
    5.89133129e-06]
 [ -3.75336378e-06  -1.83466135e-06   8.88721210e-06   5.47813441e-07
    7.79265215e-07   5.53264189e-07   5.75479602e-07   1.24813774e-07
   -2.31154732e-08   9.08559340e-08   5.23207858e-07   3.93383897e-07
    4.97970903e-07   6.18559553e-07   7.94990132e-07  -1.13326311e-07
   -2.78958780e-06]
 [  2.00431288e-06   2.98981615e-07   5.47813441e-07   5.11655556e-06
    1.92551347e-06   1.7047718

In [64]:
TG_inv = np.dot(T, inv(G))

In [65]:
d = np.dot(TG_inv, y_bar)
print d

ValueError: shapes (17,17) and (1,17) not aligned: 17 (dim 1) != 1 (dim 0)

In [ ]:
#should chi2 be calculated in log or linear?
#T == cov
TG_inv[np.diag_indices(TG_inv.shape[0])]+=np.ones(TG_inv.shape[0])
D = np.dot(TG_inv, T) # sadly not a faster way to compute D inverse. 
delta = d-y
chi2 = -0.5 * np.dot(delta, np.dot(inv(D), delta))

In [ ]:
print np.dot(T, np.dot(inv(G), T))

In [ ]:
print D

In [ ]:
print delta

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from itertools import cycle
colors = cycle(sns.color_palette())

In [ ]:
fig = plt.figure(figsize = (10, 10))

for mu, cov, label, color in zip([y_bar, y, np.abs(d)], [G, T, D], [r'$ \bar{y} $',r'$y$',r'$\mu$'], colors):
    plt.plot(rpoints, mu, color=color, label=label)
    y_err = np.sqrt(np.diag(cov))
    plt.fill_between(rpoints,mu+y_err,
                mu-y_err, color = color, alpha = 0.3)

plt.xlim(xmin = 0.1, xmax = 30)
plt.ylim(ymin = -2, ymax = 4)
#plt.ylim(ymin = 1e-2, ymax = 1e4)
#plt.title("OR Emulator w.r.t. %s"%y_param, fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$\log_{10}{\xi_{\rm gg}(r) }$', fontsize=25)
plt.legend(loc='best', fontsize=25)

plt.show()

In [ ]:
import colormaps as cmaps
plt.register_cmap(name='viridis', cmap=cmaps.viridis)
plt.set_cmap(cmaps.viridis)

In [ ]:
for mat in (np.log10(np.abs(T)), np.log10(np.abs(G)),np.log10(np.abs(inv(G))), np.log10(np.abs(np.dot(T, inv(G)))), np.log10(np.abs(D))):
    fig = plt.figure(figsize=(10,10))
    im = plt.imshow(mat, interpolation=None)
    plt.colorbar(im)
    plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))
im = plt.imshow(T*inv(T), interpolation=None)
plt.colorbar(im)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))
im = plt.imshow(G*inv(G), interpolation=None)
plt.colorbar(im)
plt.show()

In [ ]:
from scipy.linalg import eigh
print sorted(eigh(T, eigvals_only = True), reverse=True)
print np.sum(eigh(T, eigvals_only = True)>1e-3)

In [ ]:
print sorted(eigh(G, eigvals_only = True), reverse=True)
print np.sum(eigh(G, eigvals_only = True)>1e-3)

In [ ]:
np.dot(T, np.dot(inv(G), T))

In [ ]:
T